In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from stgcn_model import STGCN

keypoints = [
    "Point_0", "Point_7", "Point_8", "Point_11", "Point_12", "Point_13",
    "Point_14", "Point_15", "Point_16", "Point_17", "Point_18", "Point_21",
    "Point_22","Point_23", "Point_24", "Point_25", "Point_26", "Point_27",
    "Point_28", "Point_29", "Point_30"
]

def load_json_skeleton(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    num_frames = len(data["frames"])
    num_joints = len(keypoints)
    num_features = 2  # (x, y)
    num_views = 2

    # ✅ (1, 프레임, 뷰, 관절, 좌표) 형태로 데이터 배열 생성
    X_data = np.zeros((1, num_frames, num_views, num_joints, num_features), dtype=np.float32)

    views = ["view1", "view3"]

    # ✅ JSON 데이터 -> 배열 변환
    for frame_idx, frame in enumerate(data["frames"]):
        for view_idx, view in enumerate(views):
            pts = frame.get(view, {}).get("pts", {})
            for joint_idx, joint_name in enumerate(keypoints):
                if joint_name in pts:
                    X_data[0, frame_idx, view_idx, joint_idx, 0] = pts[joint_name]["x"]
                    X_data[0, frame_idx, view_idx, joint_idx, 1] = pts[joint_name]["y"]

    return X_data
        
class PushUpPostureAnalyzer:
    def __init__(self, model):
        """
        ST-GCN 모델을 활용한 푸쉬업 자세 분석기.
        """
        self.model = model
        self.joint_indices = {
            "head": 0,  # 머리 (코)
            "upper_back": 11,  # 어깨 (왼쪽)
            "lower_back": 23,  # 골반 (왼쪽)
            "shoulder": 11,  # 어깨 (왼쪽)
            "elbow": 13,  # 팔꿈치 (왼쪽)
            "wrist": 15,  # 손목 (왼쪽)
            "left_wrist": 15,
            "right_wrist": 16,
            "left_elbow": 13,
            "right_elbow": 14,
            "left_hip": 23,
            "right_hip": 24,
            "left_knee": 25,
            "right_knee": 26,
            "left_ankle": 27,
            "right_ankle": 28,
            "chest": 11  # 가슴 (왼쪽 어깨로 대체)
        }
    
    def detect_faulty_posture(self, skeleton_sequence):
        """푸쉬업 동작을 분석하고 잘못된 자세를 감지합니다."""
        predictions = self.model.predict(skeleton_sequence)
        predicted_label = np.argmax(predictions)
        faults = {}
        
        if predicted_label == 1:  # 잘못된 자세로 분류된 경우
            faults["척추"] = self.check_neutral_spine(skeleton_sequence)
            faults["팔꿈치"] = self.check_elbow_angle(skeleton_sequence)
            faults["가슴"] = self.check_chest_movement(skeleton_sequence)
            faults["손 위치"] = self.check_hand_position(skeleton_sequence)
            faults["머리 정렬"] = self.check_head_alignment(skeleton_sequence)
        
        return {k: v for k, v in faults.items() if v is not None}
    
    def check_neutral_spine(self, skeleton_sequence):
        """척추가 중립적인 상태를 유지하는지 확인합니다."""
        spine_joints = [self.joint_indices['upper_back'], self.joint_indices['lower_back']]
        
        # 기존 각도 계산 방식 대신 벡터 기울기 활용
        upper_back = skeleton_sequence[:, spine_joints[0], :]
        lower_back = skeleton_sequence[:, spine_joints[1], :]
        
        spine_vector = upper_back - lower_back  # 두 점을 연결하는 벡터
        spine_angle = np.arctan2(spine_vector[:, 1], spine_vector[:, 0]) * (180 / np.pi)  # 라디안 → 도 단위 변환
        
        if np.any(np.abs(spine_angle - 90) > 15):  # 수직에서 ±15도 이상 벗어나면 경고
            return "척추가 중립적이지 않습니다. 허리를 곧게 펴세요."
        return None

    
    def check_elbow_angle(self, skeleton_sequence):
        """팔꿈치가 최저점에서 90도를 이루는지 확인합니다."""
        elbow_joints = [self.joint_indices['shoulder'], self.joint_indices['elbow'], self.joint_indices['wrist']]
        elbow_angles = self.calculate_joint_angle(skeleton_sequence, elbow_joints)
        
        if np.min(elbow_angles) > 100:
            return "팔꿈치가 충분히 구부러지지 않았습니다. 90도까지 구부리세요."
        return None
    
    def check_chest_movement(self, skeleton_sequence):
        """가슴이 충분히 아래로 내려가는지 확인합니다."""
        chest_index = self.joint_indices['chest']
        movement_range = np.max(skeleton_sequence[:, chest_index, 1]) - np.min(skeleton_sequence[:, chest_index, 1])
        
        if movement_range < 0.2:
            return "가슴이 충분히 내려가지 않았습니다. 몸을 더 낮추세요."
        return None
    
    def check_hand_position(self, skeleton_sequence):
        """손의 위치가 가슴과 일직선상에 있는지 확인합니다."""
        wrist_indices = [self.joint_indices['left_wrist'], self.joint_indices['right_wrist']]
        chest_index = self.joint_indices['chest']
        
        hand_positions = skeleton_sequence[:, wrist_indices, 0]
        chest_position = skeleton_sequence[:, chest_index, 0]
        
        if np.any(np.abs(hand_positions - chest_position) > 0.2):
            return "손이 가슴과 정렬되지 않았습니다. 손의 위치를 조정하세요."
        return None
    
    def check_head_alignment(self, skeleton_sequence):
        """머리가 바르게 정렬되어 있는지 확인합니다."""
        head_index = self.joint_indices['head']
        neck_index = self.joint_indices['upper_back']
        
        head_movement = np.abs(skeleton_sequence[:, head_index, 1] - skeleton_sequence[:, neck_index, 1])
        if np.any(head_movement > 0.1):
            return "머리 위치가 올바르지 않습니다. 머리를 중립적으로 유지하세요."
        return None
    
    def calculate_joint_angle(self, skeleton_sequence, joint_indices):
        """세 개의 관절을 이용해 각도를 계산합니다."""
        a, b, c = [skeleton_sequence[:, idx, :] for idx in joint_indices]
        
        ab = a - b
        bc = c - b
        
        ab_norm = np.linalg.norm(ab, axis=1)
        bc_norm = np.linalg.norm(bc, axis=1)
        dot_product = np.sum(ab * bc, axis=1)
        
        angles = np.arccos(dot_product / (ab_norm * bc_norm)) * (180 / np.pi)
        return angles
    
    def provide_feedback(self, skeleton_sequence):
        """감지된 자세 오류를 기반으로 실시간 피드백을 제공합니다."""
        faults = self.detect_faulty_posture(skeleton_sequence)
        
        if not faults:
            return "훌륭합니다! 푸쉬업 자세가 올바릅니다."
        
        feedback = "다음 사항을 수정하세요: "
        for key, message in faults.items():
            feedback += f"\n- {message}"
        
        return feedback

# 모델 로드
model = load_model("stgcn_model6.keras", custom_objects={"STGCN": STGCN})
analyzer = PushUpPostureAnalyzer(model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
훌륭합니다! 푸쉬업 자세가 올바릅니다.


In [22]:
skeleton_sequence = load_json_skeleton("D:/Studying/gradu/013.피트니스자세/2.Validation/검증데이터/body_v-1-566.json")
feedback = analyzer.provide_feedback(skeleton_sequence)
print(feedback)
skeleton_sequence = load_json_skeleton("D:/Studying/gradu/013.피트니스자세/2.Validation/검증데이터/body_v-1-567.json")
feedback = analyzer.provide_feedback(skeleton_sequence)
print(feedback)
skeleton_sequence = load_json_skeleton("D:/Studying/gradu/013.피트니스자세/2.Validation/검증데이터/body_v-1-568.json")
feedback = analyzer.provide_feedback(skeleton_sequence)
print(feedback)
skeleton_sequence = load_json_skeleton("D:/Studying/gradu/013.피트니스자세/2.Validation/검증데이터/body_v-1-569.json")
feedback = analyzer.provide_feedback(skeleton_sequence)
print(feedback)
skeleton_sequence = load_json_skeleton("D:/Studying/gradu/013.피트니스자세/2.Validation/검증데이터/body_v-1-570.json")
feedback = analyzer.provide_feedback(skeleton_sequence)
print(feedback)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
훌륭합니다! 푸쉬업 자세가 올바릅니다.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
훌륭합니다! 푸쉬업 자세가 올바릅니다.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
훌륭합니다! 푸쉬업 자세가 올바릅니다.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
훌륭합니다! 푸쉬업 자세가 올바릅니다.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
훌륭합니다! 푸쉬업 자세가 올바릅니다.
